In [2]:
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt

In [4]:
DATA_ROOT_DIR="FashionMNIST"

In [10]:
train_data=torchvision.datasets.FashionMNIST(
root=DATA_ROOT_DIR,
train=True,
download=True,
transform=torchvision.transforms.ToTensor())
test_data=torchvision.datasets.FashionMNIST(
root=DATA_ROOT_DIR,
train=False,
download=True,
transform=torchvision.transforms.ToTensor())
"""
ToTensor()->basically scalling down from 0 to 1
"""

'\nToTensor()->basically scalling down from 0 to 1\n'

'\ntrain_data_loader-> its a methodand its actually create an python iterator. Its load our data and its take care of all our parallel processing like how you load the data and how you shuffling.\n\n'

In [8]:
BATCH_SIZE=64
"""
train_data_loader-> its a methodand its actually create an python iterator. Its load our data and its take care of all our parallel processing like how you load the data and how you shuffling.

"""
train_data_loader=torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE)
test_data_loader=torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

In [12]:
for X, y in test_data_loader:
    print(X.shape)
    #For all of the images  we have respective classe label. we can check for that torch.unique(y)
    print(y.shape, y.dtype, torch.unique(y))
    break

torch.Size([64, 1, 28, 28])
torch.Size([64]) torch.int64 tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [13]:
device="cuda" if torch.cuda.is_available() else 'cpu'
device

'cpu'

# Lets create a NN

In [14]:
# nn.Module inherites the basic nn module
class NeuralNetwork(nn.Module):
    # NeuralNetwork is a child class for nn Module
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.flatten=nn.Flatten()
        
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
            nn.ReLU()
        )
    def forward(self, X):#Forward propagation  X->input or train data
        X=self.flatten(X)
        logits=self.linear_relu_stack(X)
        return logits